# Step 1: Check Nominal Parameters

In [10]:
from ENDFtk.tree import Tape
import numpy as np
import glob
import matplotlib.pyplot as plt

tape = Tape.from_file('/home/sole-pie01/ndlib/tendl23-neutron/n-Fe056.tendl.txt')
mf42 = tape.MAT(tape.material_numbers[0]).MF(4).MT(2).parse()
mf342 = tape.MAT(tape.material_numbers[0]).MF(34).MT(2).parse()
sampled_params = []

mf42.distributions.legendre.angular_distributions[0].legendre_order
# mf342.reactions[0].legendre_blocks[0].data[0].energies[-1]

# mf42.distributions.legendre.incident_energies[-1]
# for index, distrib in enumerate(mf42.distributions.legendre.angular_distributions):
#     print(f"Number of orders={distrib.legendre_order} at energy={distrib.E}")
#     print(distrib.coefficients[:])
#     for j, spingparity in enumerate(orbital.j_values):
#         print(f"Orbital, l={l}, J={j}, pi={spingparity.spin}")
#         print(f"  {len(spingparity.energies[:])} Energies: {spingparity.energies[:]}")
#         print(f"  Spacings: {spingparity.average_level_spacings[:]}")
#         sampled_params.append(spingparity.average_level_spacings[:])
#         # print(f"  Gamma widths: {spingparity.average_gamma_widths[:]}")
#         print(f"  Neutron widths: {spingparity.average_neutron_widths[:]}")
#         sampled_params.append(spingparity.average_neutron_widths[:])
        
# len(sampled_params)


2

In [15]:
import numpy as np
import matplotlib.pyplot as plt

def mesh_union(mesh1, mesh2, eps=1e-8):
    union = np.unique(np.concatenate((mesh1, mesh2)))
    diff = np.diff(union)
    mask = diff < eps
    if np.any(mask):
        keep = np.ones_like(union, dtype=bool)
        keep[1:][mask] = False
        union = union[keep]
    return union

def expand_matrix_fast(original_matrix, original_row_mesh, original_col_mesh, union_row_mesh, union_col_mesh):
    original_row_mesh = np.array(sorted(original_row_mesh))
    original_col_mesh = np.array(sorted(original_col_mesh))
    union_row_mesh = np.array(sorted(union_row_mesh))
    union_col_mesh = np.array(sorted(union_col_mesh))

    original_row_size = len(original_row_mesh) - 1
    original_col_size = len(original_col_mesh) - 1

    row_indices = np.searchsorted(original_row_mesh, union_row_mesh[:-1], side='right') - 1
    col_indices = np.searchsorted(original_col_mesh, union_col_mesh[:-1], side='right') - 1

    row_indices = np.clip(row_indices, 0, original_row_size-1)
    col_indices = np.clip(col_indices, 0, original_col_size-1)

    expanded_matrix = original_matrix[np.ix_(row_indices, col_indices)]
    return expanded_matrix

def add_matrices_with_mesh(matrixA, rowMeshA, colMeshA, matrixB, rowMeshB, colMeshB, epsilon=1e-8):
    if matrixA.size == 0:
        return matrixB.copy(), sorted(rowMeshB), sorted(colMeshB)
    if matrixB.size == 0:
        return matrixA.copy(), sorted(rowMeshA), sorted(colMeshA)

    rowMeshA = np.array(sorted(rowMeshA))
    colMeshA = np.array(sorted(colMeshA))
    rowMeshB = np.array(sorted(rowMeshB))
    colMeshB = np.array(sorted(colMeshB))

    union_row_mesh = mesh_union(rowMeshA, rowMeshB, epsilon)
    union_col_mesh = mesh_union(colMeshA, colMeshB, epsilon)

    expandedA = expand_matrix_fast(matrixA, rowMeshA, colMeshA, union_row_mesh, union_col_mesh)
    expandedB = expand_matrix_fast(matrixB, rowMeshB, colMeshB, union_row_mesh, union_col_mesh)

    result = expandedA + expandedB
    return result, union_row_mesh.tolist(), union_col_mesh.tolist()

def subblock_to_matrix(subblock):
    # LB==5: symmetric, upper triangle stored
    if hasattr(subblock, "LB") and subblock.LB == 5:
        N = subblock.NE - 1
        mesh = subblock.energies.to_list()
        mat = np.zeros((N, N))
        triu_indices = np.triu_indices(N)
        mat[triu_indices] = subblock.values.to_list()
        mat = mat + mat.T - np.diag(np.diag(mat))
        return mat, mesh, mesh
    # LB==1: diagonal
    elif hasattr(subblock, "LB") and subblock.LB == 1:
        mesh = subblock.first_array_energies.to_list()
        vals = subblock.first_array_fvalues.to_list()
        mat = np.diag(vals)
        return mat, mesh, mesh
    # CovariancePairs (LB==1)
    elif hasattr(subblock, "number_pairs"):
        mesh = subblock.first_array_energies.to_list()
        vals = subblock.first_array_fvalues.to_list()
        mat = np.diag(vals)
        return mat, mesh, mesh
    else:
        raise NotImplementedError("Unknown subblock type")

def block_to_matrix(block):
    # block is ENDFtk.SquareMatrix or ENDFtk.LegendreBlock
    # block.data.to_list() gives subblocks
    subblocks = block.data.to_list() if hasattr(block, "data") else [block]
    matrix = np.zeros((0,0))
    row_mesh = []
    col_mesh = []
    for sub in subblocks:
        submat, subrow, subcol = subblock_to_matrix(sub)
        if matrix.size == 0:
            matrix = submat
            row_mesh = subrow
            col_mesh = subcol
        else:
            matrix, row_mesh, col_mesh = add_matrices_with_mesh(
                matrix, row_mesh, col_mesh, submat, subrow, subcol
            )
    return matrix, row_mesh, col_mesh

def covariance_to_correlation_and_relstd(rel_cov):
    diag = np.diag(rel_cov)
    std = np.sqrt(np.maximum(diag, 0))
    # Avoid division by zero
    with np.errstate(divide='ignore', invalid='ignore'):
        corr = rel_cov / np.outer(std, std)
        corr[~np.isfinite(corr)] = 0.0
        relstd = std.copy()
    return corr, relstd

def retrieve_full_covariance_matrix(mt2):
    NL = mt2.NL
    NL1 = mt2.NL1
    nblocks = mt2.number_legendre_blocks
    blocks = mt2.legendre_blocks.to_list()
    # First, collect all unique energy mesh points for all blocks
    all_mesh = set()
    for block in blocks:
        for sub in block.data.to_list():
            if hasattr(sub, "LB") and sub.LB == 5:
                all_mesh.update(sub.energies.to_list())
            elif hasattr(sub, "LB") and sub.LB == 1:
                all_mesh.update(sub.first_array_energies.to_list())
            elif hasattr(sub, "number_pairs"):
                all_mesh.update(sub.first_array_energies.to_list())
    all_mesh = sorted(all_mesh)
    N = len(all_mesh) - 1

    # Prepare the full relative covariance matrix
    full_rel_cov = np.zeros((NL*N, NL1*N))
    # For each block (l, l1), fill the corresponding submatrix
    for idx, block in enumerate(blocks):
        l = block.L
        l1 = block.L1
        mat, row_mesh, col_mesh = block_to_matrix(block)
        # Expand to the global mesh
        mat_expanded = expand_matrix_fast(mat, row_mesh, col_mesh, all_mesh, all_mesh)
        # Place in the full matrix
        full_rel_cov[(l-1)*N:l*N, (l1-1)*N:l1*N] = mat_expanded
        if l != l1:
            # Fill symmetric block
            full_rel_cov[(l1-1)*N:l1*N, (l-1)*N:l*N] = mat_expanded.T

    return full_rel_cov
mt2 = mf342.reactions.to_list()[0]
relativ_cov = retrieve_full_covariance_matrix(mt2)
# Count non zero in the diagonal
np.count_nonzero(np.diag(relativ_cov))

14

## Step 2: Build sample matrix from perturbed tapes and verify relative covariance
We read sampled_tape_random1.endf ... sampled_tape_random9.endf, interpolate MF(4) Legendre coefficients at midpoints of the global covariance energy mesh (from MF(34)), flatten order-by-order to form a parameter vector per sample, then compare the empirical relative covariance to the MF(34) reference.


In [16]:
import os, re
from pathlib import Path
from ENDFtk.tree import Tape
import numpy as np

sampled_dir = Path('../FreeGazScattering')
pattern = re.compile(r'^sampled_tape_random(\d+)\.endf$')

sample_files = []
if sampled_dir.is_dir():
    for fname in os.listdir(sampled_dir):
        m = pattern.match(fname)
        if m:
            sample_files.append((int(m.group(1)), sampled_dir / fname))

sample_files.sort()
print(f"Found {len(sample_files)} sampled tapes: {[p.name for _, p in sample_files]}")
if not sample_files:
    raise FileNotFoundError("No sampled_tape_random*.endf files found in ../FreeGazScattering")

blocks = mf342.reactions[0].legendre_blocks.to_list()
all_mesh = set()
for block in blocks:
    for sub in block.data.to_list():
        if hasattr(sub, 'LB') and sub.LB == 5:
            all_mesh.update(sub.energies.to_list())
        elif hasattr(sub, 'LB') and sub.LB == 1:
            all_mesh.update(sub.first_array_energies.to_list())
        elif hasattr(sub, 'number_pairs'):
            all_mesh.update(sub.first_array_energies.to_list())
all_mesh = sorted(all_mesh)
energy_bins = list(zip(all_mesh[:-1], all_mesh[1:]))
midpoints = np.array([0.5*(a+b) for a,b in energy_bins])
print(f"Covariance global mesh: {len(all_mesh)} points -> {len(energy_bins)} bins")

orders_present_cov = sorted({blk.L for blk in blocks})
print("Orders present (from covariance blocks):", orders_present_cov)

from bisect import bisect_right

def extract_min_common_order(tape_obj, mt=2):
    try:
        mf4_local = tape_obj.MAT(tape_obj.material_numbers[0]).MF(4).MT(mt).parse()
    except Exception as e:
        print(f"  [WARN] Failed to parse MF4/MT{mt}: {e}")
        return -1
    leg = mf4_local.distributions.legendre
    distributions = getattr(leg, 'angular_distributions', [])
    if not distributions:
        return -1
    # mf42.distributions.legendre.angular_distributions[0].legendre_order
    per_energy_actual = [dist.legendre_order + 1 for dist in distributions]
    # print(per_energy_actual)
    return min(per_energy_actual) if per_energy_actual else -1

def interpolate_legendre_coeffs(tape_obj, target_orders, mt=2, energies=midpoints):
    mf4_local = tape_obj.MAT(tape_obj.material_numbers[0]).MF(4).MT(mt).parse()
    leg = mf4_local.distributions.legendre
    E_grid = np.array(leg.incident_energies[:])
    distributions = leg.angular_distributions
    max_order_needed = max(target_orders)
    coeff_grid = np.full((len(E_grid), max_order_needed+1), np.nan)
    for i, dist in enumerate(distributions):
        coeffs = np.array(dist.coefficients[:])
        take = min(len(coeffs)-1, max_order_needed)
        if take >= 0:
            coeff_grid[i, :take+1] = coeffs[:take+1]
    result_rows = []
    for l in target_orders:
        y = coeff_grid[:, l]
        mask = ~np.isnan(y)
        if mask.sum() < 2:
            interp_vals = np.zeros_like(energies)
        else:
            E_valid = E_grid[mask]
            y_valid = y[mask]
            interp_vals = np.interp(energies, E_valid, y_valid)
        result_rows.append(interp_vals)
    arr = np.vstack(result_rows)
    return arr.reshape(len(target_orders)*len(energies))

print("Scanning sample tapes for available orders...")
nominal_min_order = extract_min_common_order(tape)
per_sample_info = []  # (idx, path, min_order)
for idx, p in sample_files:
    t = Tape.from_file(str(p))
    mo = extract_min_common_order(t)
    per_sample_info.append((idx, p, mo))
    print(f"  {p.name}: min_common_order={mo}")

valid_samples = [(i,p,mo) for i,p,mo in per_sample_info if mo >= 1]
if not valid_samples:
    print("[ERROR] None of the sampled tapes provide Legendre coefficients (order >=1). Aborting verification.")
    verification_results = dict(
        sample_files=[str(p) for _,p in sample_files],
        active_orders=[],
        note="No valid sampled tapes with MF(4) Legendre distributions."
    )
else:
    min_common_order = min([nominal_min_order] + [mo for _,_,mo in valid_samples])
    active_orders = [l for l in orders_present_cov if l <= min_common_order]
    print("Selected active orders:", active_orders)
    if not active_orders:
        print("[ERROR] No overlapping active orders between nominal and samples.")
        verification_results = dict(
            sample_files=[str(p) for _,p,_ in valid_samples],
            active_orders=[],
            note="No overlapping orders."
        )
    else:
        nominal_vec = interpolate_legendre_coeffs(tape, active_orders)
        vectors = []
        used_files = []
        for idx, path, mo in valid_samples:
            t = Tape.from_file(str(path))
            vec = interpolate_legendre_coeffs(t, active_orders)
            vectors.append(vec)
            used_files.append(path)
        sample_matrix = np.vstack(vectors)
        print(f"Sample matrix shape: {sample_matrix.shape} (using {len(used_files)} tapes)")
        abs_tol = 1e-14
        nominal_safe = nominal_vec.copy()
        small_mask = np.abs(nominal_safe) < abs_tol
        nominal_safe[small_mask] = 1.0
        rel_deltas = (sample_matrix - nominal_vec) / nominal_safe
        rel_deltas[:, small_mask] = 0.0
        empirical_rel_cov = np.cov(rel_deltas, rowvar=False, bias=False)
        N_bins = len(energy_bins)
        ref_indices = []
        for l in active_orders:
            ref_indices.extend(range((l-1)*N_bins, (l-1)*N_bins + N_bins))
        ref_indices = np.array(ref_indices)
        reference_rel_cov = relativ_cov[np.ix_(ref_indices, ref_indices)]
        ref_diag = np.diag(reference_rel_cov)
        emp_diag = np.diag(empirical_rel_cov)
        with np.errstate(divide='ignore', invalid='ignore'):
            diag_rel_error = np.where(ref_diag != 0, (emp_diag - ref_diag)/ref_diag, 0.0)
        print(f"Diagonal rel error stats: mean={np.mean(diag_rel_error):.3e}, median={np.median(diag_rel_error):.3e}, max_abs={np.max(np.abs(diag_rel_error)):.3e}")
        ref_std = np.sqrt(np.maximum(ref_diag, 0))
        emp_std = np.sqrt(np.maximum(emp_diag, 0))
        with np.errstate(divide='ignore', invalid='ignore'):
            ref_corr = reference_rel_cov / np.outer(ref_std, ref_std)
            emp_corr = empirical_rel_cov / np.outer(emp_std, emp_std)
            ref_corr[~np.isfinite(ref_corr)] = 0.0
            emp_corr[~np.isfinite(emp_corr)] = 0.0
        corr_rel_diff = np.linalg.norm(emp_corr - ref_corr) / (np.linalg.norm(ref_corr) + 1e-30)
        print(f"Correlation relative Frobenius diff: {corr_rel_diff:.3e}")
        for i, l in enumerate(active_orders):
            seg = slice(i*N_bins, (i+1)*N_bins)
            ref_d = ref_diag[seg]
            emp_d = emp_diag[seg]
            with np.errstate(divide='ignore', invalid='ignore'):
                rel_err = np.where(ref_d != 0, (emp_d - ref_d)/ref_d, 0.0)
            rms = np.sqrt(np.mean(rel_err**2)) if rel_err.size else 0.0
            print(f"Order l={l}: diag RMS rel error={rms:.3e}")
        verification_results = dict(
            sample_files=[str(p) for p in used_files],
            dropped_files=[str(p) for _,p,mo in per_sample_info if p not in used_files],
            active_orders=active_orders,
            energy_midpoints=midpoints,
            sample_matrix=sample_matrix.tolist(),
            relative_deltas=rel_deltas.tolist(),
            empirical_rel_cov=empirical_rel_cov.tolist(),
            reference_rel_cov=reference_rel_cov.tolist(),
            diag_rel_error=diag_rel_error.tolist(),
            corr_rel_frobenius_diff=float(corr_rel_diff),
        )
print("Verification results prepared.")


Found 10 sampled tapes: ['sampled_tape_random1.endf', 'sampled_tape_random2.endf', 'sampled_tape_random3.endf', 'sampled_tape_random4.endf', 'sampled_tape_random5.endf', 'sampled_tape_random6.endf', 'sampled_tape_random7.endf', 'sampled_tape_random8.endf', 'sampled_tape_random9.endf', 'sampled_tape_random10.endf']
Covariance global mesh: 5 points -> 4 bins
Orders present (from covariance blocks): [1, 2, 3, 4, 5, 6]
Scanning sample tapes for available orders...
  sampled_tape_random1.endf: min_common_order=1
  sampled_tape_random2.endf: min_common_order=1
  sampled_tape_random3.endf: min_common_order=1
  sampled_tape_random4.endf: min_common_order=1
  sampled_tape_random5.endf: min_common_order=1
  sampled_tape_random6.endf: min_common_order=1
  sampled_tape_random7.endf: min_common_order=1
  sampled_tape_random8.endf: min_common_order=1
  sampled_tape_random9.endf: min_common_order=1
  sampled_tape_random10.endf: min_common_order=1
Selected active orders: [1]
Sample matrix shape: (10, 